In [1]:
import numpy as np
import pandas as pd

# acquire
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import os

# prepare
import unicodedata
import re
import json
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# explore
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# model
from sklearn.linear_model import LogisticRegression
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import acquire
import prepare
import wrangle
import github_acquire

Acquire module loaded
credentials loaded successfully
Wrangle module loaded successfully.
git acquire module loaded successsfully


In [3]:
# df_raw = pd.read_json('repos.json')
# df.head()

In [4]:
# len(df_raw.language[df_raw.language.isnull()])

In [5]:
# importing the cleaned json dictionary of readme content.

df = pd.read_json('repos_clean.json')
df.head()

,language,repo,content,stemmed,lemmatized,clean
0,JavaScript,facebook/react,React · \nReact is a JavaScript library for...,react react is a javascript librari for build ...,react react is a javascript library for buildi...,react react javascript library building user i...
1,JavaScript,d3/d3,D3: Data-Driven Documents\n\nD3 (or D3.js) is ...,d3 datadriven document d3 or d3j is a javascri...,d3 datadriven document d3 or d3js is a javascr...,d3 datadriven document d3 d3js javascript libr...
2,JavaScript,vuejs/vue,\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue.j...,support vuej vuej is an mitlicens open sourc p...,supporting vuejs vuejs is an mitlicensed open ...,supporting vuejs vuejs mitlicensed open source...
3,JavaScript,axios/axios,axios\n\n\n\n\n\n\n\n\nPromise based HTTP clie...,axio promis base http client for the browser a...,axios promise based http client for the browse...,axios promise based http client browser nodejs...
4,JavaScript,facebook/create-react-app,Create React App \n\nCreate React apps with n...,creat react app creat react app with no build ...,create react app create react apps with no bui...,create react app create react apps build confi...


In [6]:
# # add a column that is a list of each word for each repo --> PREPARE.PY
# words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df.clean] 

# # column name will be words, and the column will contain lists of the words in each doc
# df = pd.concat([df, pd.DataFrame({'words': words})], axis=1)
# df.head()

### Splitting the data

In [9]:
wrangle.train_validate_test?

In [10]:
X_train, y_train, X_validate, y_validate, X_test, y_test, train_explore, df_explore = wrangle.train_validate_test(df, 'language')

In [11]:
X_train.head()

,repo,content,stemmed,lemmatized,clean
90,select2/select2,Select2\n\n\n\n\nSelect2 is a jQuery-based rep...,select2 select2 is a jquerybas replac for sele...,select2 select2 is a jquerybased replacement f...,select2 select2 jquerybased replacement select...
254,plotly/dash,Dash\n\n\n\n\n\n\n\nDash is the most downloade...,dash dash is the most download trust python fr...,dash dash is the most downloaded trusted pytho...,dash dash downloaded trusted python framework ...
89,NervJS/taro,Taro\n\n\n\n\n\n\n\n👽 Taro['tɑ:roʊ]，泰罗·奥特曼，宇宙警...,taro tarotro taro reactvuenerv qq h5 webreact ...,taro tarotro taro reactvuenerv qq h5 webreact ...,taro tarotro taro reactvuenerv qq h5 webreact ...
422,DIYgod/RSSHub,\n\n\nRSSHub\n\n🍰 Everything is RSSible\n\n\n\...,rsshub everyth is rssibl introduct rsshub is a...,rsshub everything is rssible introduction rssh...,rsshub everything rssible introduction rsshub ...
61,gulpjs/gulp,\n\n\n\nThe streaming build system\n\n ...,the stream build system what is gulp autom gul...,the streaming build system what is gulp automa...,streaming build system gulp automation gulp to...


#### Bag of Words Setup

In [12]:
# Create CountVectorizer, which create bag-of-words model.
# stop_words : Specify language to remove stopwords. 
# min_df: ignore terms that have a document frequency strictly 
# lower than the given threshold. This value is also called cut-off in the literature. 
# If float, the parameter represents a proportion of documents, integer absolute counts. 
# ngram_range: the lower and upper boundary of the range of n-values for 
# different word n-grams or char n-grams to be extracted. 

vectorizer = CountVectorizer(stop_words='english', 
                             min_df=20, 
                             ngram_range=(1,2), 
                             binary=True)

# Learn vocabulary in sentences. 
vectorizer.fit(X_train.clean)

# Get dictionary. 
vectorizer.get_feature_names()

['10',
 '100',
 '1000',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '19',
 '20',
 '200',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '21',
 '22',
 '23',
 '24',
 '25',
 '27',
 '30',
 '32',
 '34',
 '35',
 '36',
 '37',
 '40',
 '50',
 'ability',
 'able',
 'absolute',
 'accept',
 'accepts',
 'access',
 'accessible',
 'according',
 'account',
 'action',
 'active',
 'actual',
 'actually',
 'add',
 'added',
 'adding',
 'addition',
 'additional',
 'additionally',
 'address',
 'advanced',
 'affect',
 'ai',
 'aim',
 'algorithm',
 'alias',
 'allow',
 'allowing',
 'allows',
 'alternative',
 'alternatively',
 'analysis',
 'andor',
 'android',
 'angular',
 'animation',
 'answer',
 'apache',
 'api',
 'apis',
 'app',
 'application',
 'applied',
 'apply',
 'approach',
 'appropriate',
 'apps',
 'aptget',
 'architecture',
 'area',
 'arent',
 'argument',
 'array',
 'article',
 'ask',
 'ask question',
 'asked',
 'associated',
 'async',
 'asynchronous',
 'attempt',
 'attribute',
 'author',
 

In [20]:
# Transform each sentences in vector space.
bow = vectorizer.transform(X_train.clean)

# this is just to see the array of 0's and 1's
bow_array = bow.toarray()
print(bow_array[1])
print(len(bow_array[1]))

[0 0 0 ... 0 0 0]
1113


In [15]:
# Show sentences and vector space representation.
# purely to visualize what's happening.
for i, v in zip(X_train.clean, bow_array):
    print(i)
    print(v)

select2 select2 jquerybased replacement select box support searching remote data set pagination result get started checkout example documentation httpsselect2org use case enhancing native selects search enhancing native selects better multiselect interface loading data javascript easily load item via ajax searchable nesting optgroups native selects support one level nesting select2 restriction tagging ability add new item fly working large remote datasets ability partially load dataset based search term paging large datasets easy support loading page result scrolled end templating support custom rendering result selection browser compatibility ie 8 chrome 8 firefox 10 safari 3 opera 106 usage source select2 directly cdn like jsdelivr cdnjs download github repo use one integration integration third party developer created plugins platform allow select2 integrated natively quickly many platform additional plugins required select2 act standard select box plugins backdrop cm backdropselect

#### TD-IDF Setup

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', min_df=20, 
                             ngram_range=(1,2), 
                             binary=True)

tfidf_sparse_matrix = tfidf.fit_transform(train.clean)
tfidf_sparse_matrix

In [ ]:
pd.DataFrame(tfidf_sparse_matrix.todense(), columns=tfidf.get_feature_names()).head()

In [ ]:
# Get vocabularies.
tfidf.vocabulary_

In [ ]:
# Transform to document-term matrix
vector_spaces = tfidf.transform(X_train.clean)
vector_spaces.toarray()

#### Creating the X_train and y_train variables for modeling:

In [ ]:
# Assigning the target:
y = train['language']

# Assigning the Features:
X_bow = bow

X_tfidf = tfidf_sparse_matrix

In [ ]:
# Predicting based on BoW:

from sklearn.linear_model import LogisticRegression

lm = LogisticRegression().fit(X_bow, y)

train['predicted'] = lm.predict(X_bow)
train.head()

In [ ]:
confusion_matrix(train.language, train.predicted)
pd.crosstab(train.language, train.predicted)

In [ ]:
print(classification_report(train.language, train.predicted))

In [ ]:
lm_tfidf = LogisticRegression().fit(X_tfidf, y)
train['pred_tfidf'] = lm_tfidf.predict(X_tfidf)

In [ ]:
print(classification_report(train.language, train.pred_tfidf))

In [ ]:
validate.shape